In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import re
import os
tf.__version__
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
text_data = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/train.csv")

In [3]:
def clean_data(text):
    text = text.lower()
    text = re.sub(r"[-=+*\"#@!$%^&()`<>\[\]]","",text)
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"they're","they are",text)
    text = re.sub(r"there're","there are",text)
    text = re.sub(r"there's","there is",text)
    text = re.sub(r"how're","how are",text)
    text = re.sub(r"what're","what are",text)
    text = re.sub(r"where're","where ",text)
    text = re.sub(r"who're","who are",text)
    text = re.sub(r"that're","that are",text)
    text = re.sub(r"when're","when are",text)
    text = re.sub(r"how's","how is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"who's","who is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"when's","when is",text)
    text = re.sub(r"won't","would not",text)
    text = re.sub(r"nt't","can not",text)
    text = re.sub(r"\'bout'","about",text)
    text = re.sub(r"\'till'","untill",text)
    text = re.sub(r"\'ll","will",text)
    text = re.sub(r"\'ve","have",text)
    text = re.sub(r"\'re","are",text)
    text = re.sub(r"\'d","would",text)
    text = re.sub(r"\."," ",text)
    text = re.sub(r"\,"," ",text)
    text = re.sub(r"\!"," ",text)
    text = re.sub(r"\?"," ",text)
    text = re.sub(r"\;"," ",text)
    text = re.sub(r"\:"," ",text)
    return text

    
text_data['text'] = text_data['text'].apply(clean_data)
text_data['augmented_text'] = text_data['augmented_text'].apply(clean_data)




In [5]:
text_data

,text,augmented_text
0,1 billion darling harbour redevelopment plans ...,1 billion darling hsarfbour redeevelopment p3...
1,1 billion desalination claim frivolous walsh,1 ibllion desalination lciam rfivloous walsh
2,1 billion wiped from nsw budget in accounting ...,1 billino wiped from nsw budget in accountngi...
3,1 million reward in hunt for bowraville killer,1 million reward in htun for boawrville killre
4,1 million wall to shield coal train noise,1 million wqli to shjele coal raib noise
...,...,...
768636,zygier arrested after leaking mossad work to a...,zygier arrectrd after leak9nh mossad eoek to a...
768637,zygier sabotaged mission to retrieve soldiers,zgyire sabotaged misisno to retrieve soldiers
768638,zylvester streaks field in alice cup,zylvester streaks field in ialce cup
768639,zynga adoption pushes bitcoin back over 1000,z6ngz adoption pushes fitcokn naxk over 1000


In [6]:
testing = text_data[72001:72006]
text_data = text_data.iloc[0:72000,]


In [7]:

g_text_tokenizer = Tokenizer()
g_text_tokenizer.fit_on_texts(text_data['augmented_text'])
g_text_word_index = g_text_tokenizer.word_index
train_sequences = g_text_tokenizer.texts_to_sequences(text_data['augmented_text'])
train_padded = pad_sequences(train_sequences, padding='post')

max_length_g = train_padded.shape[1]

text_tokenizer = Tokenizer()
text_tokenizer.fit_on_texts(text_data['text'])
text_word_index = text_tokenizer.word_index
test_sequences = text_tokenizer.texts_to_sequences(text_data['text'])
test_padded = pad_sequences(test_sequences, padding='post')
max_length = test_padded.shape[1]




In [8]:
print(max_length_g,max_length)


14 14


In [9]:
# temp_padded = pad_sequences(train_padded,max_length )
# temp_padded = temp_padded.reshape((-1,test_padded.shape[-2],1))
# temp_padded.shape
train_padded.shape

(72000, 14)

In [10]:
g_vocab_size = len(g_text_word_index)+1
vocab_size = len(text_word_index)+1
print(g_vocab_size, vocab_size)

154250 31997


In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=g_vocab_size, output_dim=1028, input_length=max_length_g, mask_zero=True),
    tf.keras.layers.LSTM(1028),
    tf.keras.layers.RepeatVector(max_length),
    tf.keras.layers.LSTM(1028,return_sequences=True),
    tf.keras.layers.LSTM(1028,return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(vocab_size,activation='softmax')
])
learning_rate = 0.001
optimizer = tf.keras.optimizers.RMSprop(learning_rate)
optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=['accuracy'])

In [13]:
model.summary(expand_nested=True,
    show_trainable=True)

Model: "sequential"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 embedding (Embedding)       (None, 14, 1028)          15856900  Y          
                                                       0                    
                                                                            
 lstm (LSTM)                 (None, 1028)              8458384   Y          
                                                                            
 repeat_vector (RepeatVector  (None, 14, 1028)         0         Y          
 )                                                                          
                                                                            
 lstm_1 (LSTM)               (None, 14, 1028)          8458384   Y          
                                                                            
 lstm_2 (LSTM)               (None, 14, 1028)          8

In [14]:
temp_test_padded = test_padded.reshape((test_padded.shape[0], test_padded.shape[1],1))
temp_train_padded = train_padded.reshape((train_padded.shape[0], train_padded.shape[1],1))


In [15]:
epoch = 20
model.fit(train_padded, test_padded, batch_size=128, epochs=epoch, validation_split=0.2)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:439: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 158569000 elements. This may consume a large amount of memory.
  num_elements)


450/450 [==============================] - 114s 237ms/step - loss: 4.2438 - accuracy: 0.5077 - val_loss: 4.3993 - val_accuracy: 0.5238
Epoch 2/20
450/450 [==============================] - 104s 232ms/step - loss: 3.8221 - accuracy: 0.5283 - val_loss: 4.1964 - val_accuracy: 0.5282
Epoch 3/20
450/450 [==============================] - 104s 232ms/step - loss: 3.5016 - accuracy: 0.5596 - val_loss: 3.9509 - val_accuracy: 0.5579
Epoch 4/20
450/450 [==============================] - 104s 232ms/step - loss: 3.1882 - accuracy: 0.5930 - val_loss: 3.7973 - val_accuracy: 0.5735
Epoch 5/20
450/450 [==============================] - 104s 232ms/step - loss: 2.9062 - accuracy: 0.6181 - val_loss: 3.7109 - val_accuracy: 0.5886
Epoch 6/20
450/450 [==============================] - 104s 232ms/step - loss: 2.6488 - accuracy: 0.6421 - val_loss: 3.6436 - val_accuracy: 0.6050
Epoch 7/20
450/450 [==============================] - 104s 232ms/step - loss: 2.4249 - accuracy: 0.6655 - val_loss: 3.4494 - val_accura

In [24]:
model.save('/content/drive/MyDrive/Colab_Notebooks/20_epoch_accuracy_085_Emb_bidirectional_lstm_rnn')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab_Notebooks/20_epoch_accuracy_085_Emb_bidirectional_lstm_rnn/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab_Notebooks/20_epoch_accuracy_085_Emb_bidirectional_lstm_rnn/assets


In [17]:
test_sample = text_data['augmented_text'][:2]
test_sample = g_text_tokenizer.texts_to_sequences(test_sample)
test_sample = pad_sequences(test_sample, maxlen=max_length_g, padding='post')


In [18]:
pre = model.predict(test_sample)
pre.shape

(2, 14, 31997)

In [ ]:
def deTokenize(logits):
    index_to_words = {id: word for word, id in text_tokenizer.word_index.items()}
    index_to_words[0] = ''
    pre_index = np.argmax(logits, 1)
    return ' '.join(index_to_words[prediction] for prediction in pre_index)

In [19]:
text_data['augmented_text'][0]


' 1 billion darling hsarfbour redeevelopment p3lanbs unveiled'

In [21]:
deTokenize(pre[0])

'1 billion darling harbour payment plans unveiled       '